# 2 - Test particles

Let us explore some options of the models [Vlasov](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_toy.html#struphy.models.toy.Vlasov), and [GuidingCenter](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_toy.html#struphy.models.toy.GuidingCenter). In particular, we will

1. Change the geometry
2. Change the loading of the markers
3. Set a static background magnetic field

## Particles in a cylinder

As in Tutorial 1, we shall re-create the parameter files in the notebook for the purpose of discussion. The default parameter files can be created from the console via

```
struphy params Vlasov
struphy params GuidingCenter 
```

This time, we set the simulation domain $\Omega$ to be a cylinder. We explore two options for drawing markers in posittion space:

- uniform in logical space $[0, 1]^3 = F^{-1}(\Omega)$
- uniform on the cylinder $\Omega$

We start with the generic imports:

In [ ]:
from struphy import main
from struphy.fields_background import equils
from struphy.geometry import domains
from struphy.initial import perturbations
from struphy.io.options import BaseUnits, DerhamOptions, EnvironmentOptions, FieldsBackground, Time
from struphy.kinetic_background import maxwellians

# import model, set verbosity
from struphy.models.toy import Vlasov
from struphy.pic.utilities import (
    BinningPlot,
    BoundaryParameters,
    KernelDensityPlot,
    LoadingParameters,
    WeightsParameters,
)
from struphy.topology import grids

We shall create two simulations, which we store in different output folders, defined throught the environment variables:

In [ ]:
# environment options
env = EnvironmentOptions()
env_2 = EnvironmentOptions(sim_folder="sim_2")

The other generic options will be the same for both simulations. Here, we just perform one time step und load a cylindrical geometry:

In [ ]:
# units
base_units = BaseUnits()

# time stepping
time_opts = Time(dt=0.2, Tend=0.2)

# geometry
a1 = 0.0
a2 = 5.0
Lz = 20.0
domain = domains.HollowCylinder(a1=a1, a2=a2, Lz=Lz)

We can already look a t the simulation domain:

In [ ]:
domain.show()

We can leave the equilibrium, grid and Derham complex empty:

In [ ]:
# fluid equilibrium (can be used as part of initial conditions)
equil = None

# grid
grid = grids.TensorProductGrid()

# derham options
derham_opts = DerhamOptions()

For each simulation, we must create the light-weight model instance and set parameters:

In [ ]:
# light-weight model instance
model = Vlasov()
model_2 = Vlasov()

# species parameters
model.kinetic_ions.set_phys_params()
model_2.kinetic_ions.set_phys_params()

For the second simulation, in the parameters for particle loading we choose `spatial="disc"` in order to draw uniformly on the cross section of the cylinder: 

In [ ]:
loading_params = LoadingParameters(Np=1000)
loading_params_2 = LoadingParameters(Np=1000, spatial="disc")

weights_params = WeightsParameters()
boundary_params = BoundaryParameters()

model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params
)
model_2.kinetic_ions.set_markers(
    loading_params=loading_params_2, weights_params=weights_params, boundary_params=boundary_params
)

model.kinetic_ions.set_sorting_boxes()
model_2.kinetic_ions.set_sorting_boxes()

model.kinetic_ions.set_save_data(n_markers=1.0)
model_2.kinetic_ions.set_save_data(n_markers=1.0)

Propagator options and initial conditions shall be the same in both simulations:

In [ ]:
# propagator options
model.propagators.push_vxb.options = model.propagators.push_vxb.Options()
model.propagators.push_eta.options = model.propagators.push_eta.Options()

model_2.propagators.push_vxb.options = model_2.propagators.push_vxb.Options()
model_2.propagators.push_eta.options = model_2.propagators.push_eta.Options()

In [ ]:
# initial conditions (background + perturbation)
perturbation = None
background = maxwellians.Maxwellian3D(n=(1.0, perturbation))

model.kinetic_ions.var.add_background(background)
model_2.kinetic_ions.var.add_background(background)

Let us now run the first simulation:

In [ ]:
verbose = False

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

And now the second simulation:

In [ ]:
main.run(
    model_2,
    params_path=None,
    env=env_2,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

We now post-process both runs, load the generated data and plot the initial particle positions on a cross section of the cylinder:

In [ ]:
import os

path = os.path.join(os.getcwd(), "sim_1")
path_2 = os.path.join(os.getcwd(), "sim_2")

main.pproc(path)
main.pproc(path_2)

In [ ]:
simdata = main.load_data(path)
simdata_2 = main.load_data(path_2)

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(10, 6))

orbits = simdata.orbits["kinetic_ions"]
orbits_uni = simdata_2.orbits["kinetic_ions"]

# orbits = simdata.pic_species["kinetic_ions"]["orbits"]
# orbits_uni = simdata_2.pic_species["kinetic_ions"]["orbits"]

plt.subplot(1, 2, 1)
plt.scatter(orbits[0, :, 0], orbits[0, :, 1], s=2.0)
circle1 = plt.Circle((0, 0), a2, color="k", fill=False)
ax = plt.gca()
ax.add_patch(circle1)
ax.set_aspect("equal")
plt.xlabel("x")
plt.ylabel("y")
plt.title("sim_1: draw uniform in logical space")

plt.subplot(1, 2, 2)
plt.scatter(orbits_uni[0, :, 0], orbits_uni[0, :, 1], s=2.0)
circle2 = plt.Circle((0, 0), a2, color="k", fill=False)
ax = plt.gca()
ax.add_patch(circle2)
ax.set_aspect("equal")
plt.xlabel("x")
plt.ylabel("y")
plt.title("sim_2: draw uniform on disc");

## Reflecting boundary conditions 

Let us now run for 50 time steps and with 15 particles in the cylinder.
Moreover, we set reflecting boundary conditions in radial direction, which in Struphy is always the logical direction $\eta_1$.

In [ ]:
time_opts = Time(dt=0.2, Tend=10.0)
loading_params = LoadingParameters(Np=15, spatial="disc")
boundary_params = BoundaryParameters(bc=("reflect", "periodic", "periodic"))

In [ ]:
# light-weight model instance
model = Vlasov()

# species parameters
model.kinetic_ions.set_phys_params()

model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params
)
model.kinetic_ions.set_sorting_boxes()
model.kinetic_ions.set_save_data(n_markers=1.0)

We still have to set the propagator options and the initial conditions:

In [ ]:
# propagator options
model.propagators.push_vxb.options = model.propagators.push_vxb.Options()
model.propagators.push_eta.options = model.propagators.push_eta.Options()

# initial conditions (background + perturbation)
perturbation = None
background = maxwellians.Maxwellian3D(n=(1.0, perturbation))

model.kinetic_ions.var.add_background(background)

We can now run the simulation, then post-process the data and plot the resulting orbits:

In [ ]:
verbose = False

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

In [ ]:
path = os.path.join(os.getcwd(), "sim_1")
main.pproc(path)

In [ ]:
simdata = main.load_data(path)

Under `simdata.orbits[<species_name>]` one finds a three-dimensional numpy array; the first index refers to the time step, the second index to the particle and the third index to the particel attribute. The first three attributes are the partciel positions, followed by the velocities and the (initial and time-dependent) weights.

In [ ]:
orbits = simdata.orbits["kinetic_ions"]

Nt = simdata.Nt["kinetic_ions"]
Np = simdata.Np["kinetic_ions"]
Nattr = simdata.Nattr["kinetic_ions"]

In [ ]:
import numpy as np

fig = plt.figure()
ax = fig.gca()

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

# create alpha for color scaling
Tend = time_opts.Tend
alpha = np.linspace(1.0, 0.0, Nt + 1)

# loop through particles, plot all time steps
for i in range(Np):
    ax.scatter(orbits[:, i, 0], orbits[:, i, 1], c=colors[i % 4], alpha=alpha)

circle1 = plt.Circle((0, 0), a2, color="k", fill=False)

ax.add_patch(circle1)
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(f"{Nt - 1} time steps (full color at t=0)");

## Particles in a cylinder with a magnetic field

Let us add a magnetic field to the simulation. This can be done by setting an [MHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils.html#mhd-equilibria):

<!-- Let $\Omega \subset \mathbb R^3$ be a cylinder as before. Now, we search for trajectories $(\mathbf x_p, \mathbf v_p): [0,T] \to \Omega \times \mathbb R^3$, $p = 0, \ldots, N-1$ that satisfy

$$
\begin{align}
 \dot{\mathbf x}_p &= \mathbf v_p\,,\qquad && \mathbf x_p(0) = \mathbf x_{p0}\,,
 \\[2mm]
 \dot{\mathbf v}_p &= \mathbf v_p \times \mathbf B_0(\mathbf x_p) \qquad && \mathbf v_p(0) = \mathbf v_{p0}\,,
 \end{align}
$$

where $\mathbf B_0$ is a given magnetic field from an [MHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils.html#mhd-equilibria). 
In addition to the Propagator [PushEta](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushEta) for the position update, we shall use [PushVxB](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushVxB) for the velocity update. -->

In [ ]:
B0x = 0.0
B0y = 0.0
B0z = 1.0
equil = equils.HomogenSlab(B0x=B0x, B0y=B0y, B0z=B0z)

In order to project the equilibrium on the spline basis for fast evaluation in the particle kernels, we need a Derham complex:

In [ ]:
spl_kind = (False, True, True)
derham_opts = DerhamOptions(spl_kind=spl_kind)

Now we create the light-weight instance of the model and set the species options. We shall `remove` particles that hit the boundary in $\eta_1$ (radial) direction:

In [ ]:
# light-weight model instance
model = Vlasov()

# species parameters
model.kinetic_ions.set_phys_params()

loading_params = LoadingParameters(Np=20)
boundary_params = BoundaryParameters(bc=("remove", "periodic", "periodic"))
model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params
)
model.kinetic_ions.set_sorting_boxes()
model.kinetic_ions.set_save_data(n_markers=1.0)

# propagator options
model.propagators.push_vxb.options = model.propagators.push_vxb.Options()
model.propagators.push_eta.options = model.propagators.push_eta.Options()

# initial conditions (background + perturbation)
perturbation = None
background = maxwellians.Maxwellian3D(n=(1.0, perturbation))

model.kinetic_ions.var.add_background(background)

Now the usual procedure: run, post-process, load data and finally plot the orbits:

In [ ]:
# run
verbose = False

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

In [ ]:
path = os.path.join(os.getcwd(), "sim_1")
main.pproc(path)

simdata = main.load_data(path)

In [ ]:
orbits = simdata.orbits["kinetic_ions"]

Nt = simdata.Nt["kinetic_ions"]
Np = simdata.Np["kinetic_ions"]

In [ ]:
fig = plt.figure()
ax = fig.gca()

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

# create alpha for color scaling
Tend = time_opts.Tend
alpha = np.linspace(1.0, 0.0, Nt + 1)

# loop through particles, plot all time steps
for i in range(Np):
    ax.scatter(orbits[:, i, 0], orbits[:, i, 1], c=colors[i % 4], alpha=alpha)

circle1 = plt.Circle((0, 0), a2, color="k", fill=False)

ax.add_patch(circle1)
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(f"{int(Nt - 1)} time steps (full color at t=0)");

## Particles in a Tokamak equilibrium

Let us try a more complicated [MHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils.html#mhd-equilibria), namely from an ASDEX-Upgrade equilibrium stored in an EQDSK file. We instatiate an [EQDSKequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.equils.EQDSKequilibrium) with many of its default parameters, except for the density:

In [ ]:
n1 = 0.0
n2 = 0.0
na = 1.0
equil = equils.EQDSKequilibrium(n1=n1, n2=n2, na=na)

Since [EQDSKequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.equils.EQDSKequilibrium) is an [AxisymmMHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.base.AxisymmMHDequilibrium), which in turn is a [CartesianMHDequilibrium](https://struphy.pages.mpcdf.de/struphy/sections/subsections/mhd_equils_sub.html#struphy.fields_background.mhd_equil.base.CartesianMHDequilibrium), we are free to choose any mapping for the simulation (e.g. a Cuboid for Cartesian coordinates). In order to be conforming to the boundary of the equilibrium, we shall choose the [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.Tokamak) mapping:

In [ ]:
Nel = (28, 72)
p = (3, 3)
psi_power = 0.6
psi_shifts = (1e-6, 1.0)
domain = domains.Tokamak(equilibrium=equil, Nel=Nel, p=p, psi_power=psi_power, psi_shifts=psi_shifts)

The [Tokamak](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.Tokamak) domain is a [PoloidalSplineTorus](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_base.html#struphy.geometry.base.PoloidalSplineTorus), hence

$$
 \begin{align*}
 x &= R \cos(\phi)\,,
 \\
 y &= -R \sin(\phi)\,,
 \\
 z &= Z\,,
 \end{align*}
$$

between Cartesian $(x, y, z)$- and Tokamak $(R, Z, \phi)$-coordinates holds, where $(R, Z)$ spans a poloidal plane. Moreover, the Tokamak coordinates are related to general torus coordinates $(r, \theta, \phi)$ via a polar mapping in the poloidal plane:

$$
 \begin{align*}
 R &= R_0 + r \cos(\theta)\,,
 \\
 Z &= r \sin(\theta)\,,
 \\
 \phi &= \phi\,.
 \end{align*}
$$

The torus coordinates are related to Struphy logical coordinates $\boldsymbol \eta = (\eta_1, \eta_2, \eta_3) \in [0, 1]^3$ as 

$$
 \begin{align*}
 r &= a_1 + (a_2 - a_1) \eta_1\,,
 \\
 \theta &= 2\pi \eta_2\,,
 \\
 \phi &= 2\pi \eta_3\,,
 \end{align*}
$$

where $a_2 > a_1 \geq 0$ are boundaries in the radial $r$-direction.
This can be seen for instance in the [HollowTorus](https://struphy.pages.mpcdf.de/struphy/sections/subsections/domains_avail.html#struphy.geometry.domains.HollowTorus) mapping (more complicated angle parametrizations $\theta(\eta_1, \eta_2)$ are also available, but not discussed here).

Let us plot the equilibrium magnetic field strength:

1. in the poloidal plane at $\phi = 0$
2. in the top view at $z = 0$.

In [ ]:
import numpy as np

# logical grid on the unit cube
e1 = np.linspace(0.0, 1.0, 101)
e2 = np.linspace(0.0, 1.0, 101)
e3 = np.linspace(0.0, 1.0, 101)

# move away from the singular point r = 0
e1[0] += 1e-5

In [ ]:
# logical coordinates of the poloidal plane at phi = 0
eta_poloidal = (e1, e2, 0.0)
# logical coordinates of the top view at theta = 0
eta_topview_1 = (e1, 0.0, e3)
# logical coordinates of the top view at theta = pi
eta_topview_2 = (e1, 0.5, e3)

In [ ]:
# Cartesian coordinates (squeezed)
x_pol, y_pol, z_pol = domain(*eta_poloidal, squeeze_out=True)
x_top1, y_top1, z_top1 = domain(*eta_topview_1, squeeze_out=True)
x_top2, y_top2, z_top2 = domain(*eta_topview_2, squeeze_out=True)

print(f"{x_pol.shape = }")
print(f"{x_top1.shape = }")
print(f"{x_top2.shape = }")

In [ ]:
# generate two axes
fig, axs = plt.subplots(2, 1, figsize=(8, 16))
ax = axs[0]
ax_top = axs[1]

# min/max of field strength
equil.domain = domain
Bmax = np.max(equil.absB0(*eta_topview_2, squeeze_out=True))
Bmin = np.min(equil.absB0(*eta_topview_1, squeeze_out=True))
levels = np.linspace(Bmin, Bmax, 51)

# absolute magnetic field at phi = 0
im = ax.contourf(x_pol, z_pol, equil.absB0(*eta_poloidal, squeeze_out=True), levels=levels)

# absolute magnetic field at Z = 0
im_top = ax_top.contourf(x_top1, y_top1, equil.absB0(*eta_topview_1, squeeze_out=True), levels=levels)
ax_top.contourf(x_top2, y_top2, equil.absB0(*eta_topview_2, squeeze_out=True), levels=levels)

# last closed flux surface, poloidal
ax.plot(x_pol[-1], z_pol[-1], color="k")

# last closed flux surface, toroidal
ax_top.plot(x_top1[-1], y_top1[-1], color="k")
ax_top.plot(x_top2[-1], y_top2[-1], color="k")

# limiter, poloidal
ax.plot(equil.limiter_pts_R, equil.limiter_pts_Z, "tab:orange")
ax.axis("equal")
ax.set_xlabel("R")
ax.set_ylabel("Z")
ax.set_title("abs(B) at $\phi=0$")
fig.colorbar(im)
# limiter, toroidal
limiter_Rmax = np.max(equil.limiter_pts_R)
limiter_Rmin = np.min(equil.limiter_pts_R)

thetas = 2 * np.pi * e2
limiter_x_max = limiter_Rmax * np.cos(thetas)
limiter_y_max = -limiter_Rmax * np.sin(thetas)
limiter_x_min = limiter_Rmin * np.cos(thetas)
limiter_y_min = -limiter_Rmin * np.sin(thetas)

ax_top.plot(limiter_x_max, limiter_y_max, "tab:orange")
ax_top.plot(limiter_x_min, limiter_y_min, "tab:orange")
ax_top.axis("equal")
ax_top.set_xlabel("x")
ax_top.set_ylabel("y")
ax_top.set_title("abs(B) at $Z=0$")
fig.colorbar(im_top);

We now set up a simualtion of 4 specific particle orbits in this equilibrium:

In [ ]:
# light-weight model instance
model = Vlasov()

# species parameters
model.kinetic_ions.set_phys_params()

initial = (
    (0.501, 0.001, 0.001, 0.0, 0.0450, -0.04),  # co-passing particle
    (0.511, 0.001, 0.001, 0.0, -0.0450, -0.04),  # counter passing particle
    (0.521, 0.001, 0.001, 0.0, 0.0105, -0.04),  # co-trapped particle
    (0.531, 0.001, 0.001, 0.0, -0.0155, -0.04),
)

loading_params = LoadingParameters(Np=4, seed=1608, specific_markers=initial)
boundary_params = BoundaryParameters(bc=("remove", "periodic", "periodic"))
model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params, bufsize=2.0
)
model.kinetic_ions.set_sorting_boxes()
model.kinetic_ions.set_save_data(n_markers=1.0)

# propagator options
model.propagators.push_vxb.options = model.propagators.push_vxb.Options()
model.propagators.push_eta.options = model.propagators.push_eta.Options()

# initial conditions (background + perturbation)
perturbation = None
background = maxwellians.Maxwellian3D(n=(1.0, perturbation))

model.kinetic_ions.var.add_background(background)

We again need a Derham complex for the projection of the equilibirum onto the spline basis:

In [ ]:
Nel = (32, 72, 1)
grid = grids.TensorProductGrid(Nel=Nel)

p = (3, 3, 1)
spl_kind = (False, True, True)
derham_opts = DerhamOptions(p=p, spl_kind=spl_kind)

We aim to simulate 15000 time steps with a second-order splitting algorithm:

In [ ]:
time_opts = Time(dt=0.2, Tend=3000, split_algo="Strang")

verbose = False

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

In [ ]:
import os

from struphy import main

path = os.path.join(os.getcwd(), "sim_1")
main.pproc(path)

simdata = main.load_data(path)

In [ ]:
orbits = simdata.orbits["kinetic_ions"]

Nt = simdata.Nt["kinetic_ions"]
Np = simdata.Np["kinetic_ions"]

In [ ]:
import math

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

dt = time_opts.dt
Tend = time_opts.Tend

for i in range(Np):
    r = np.sqrt(orbits[:, i, 0] ** 2 + orbits[:, i, 1] ** 2)
    # poloidal
    ax.scatter(r, orbits[:, i, 2], c=colors[i % 4], s=1)
    # top view
    ax_top.scatter(orbits[:, i, 0], orbits[:, i, 1], c=colors[i % 4], s=1)

ax.set_title(f"{math.ceil(Tend / dt)} time steps")
ax_top.set_title(f"{math.ceil(Tend / dt)} time steps")
fig

## Guiding-centers in a Tokamak equilibrium

Let us run a similar test for guiding-centers:

In [ ]:
from struphy.models.toy import GuidingCenter

# light-weight model instance
model = GuidingCenter()

# species parameters
model.kinetic_ions.set_phys_params()

initial = (
    (0.501, 0.001, 0.001, -1.935, 1.72),  # co-passing particle
    (0.501, 0.001, 0.001, 1.935, 1.72),  # couner-passing particle
    (0.501, 0.001, 0.001, -0.6665, 1.72),  # co-trapped particle
    (0.501, 0.001, 0.001, 0.4515, 1.72),
)  # counter-trapped particl

loading_params = LoadingParameters(Np=4, seed=1608, specific_markers=initial)
boundary_params = BoundaryParameters(bc=("remove", "periodic", "periodic"))
model.kinetic_ions.set_markers(
    loading_params=loading_params, weights_params=weights_params, boundary_params=boundary_params, bufsize=2.0
)
model.kinetic_ions.set_sorting_boxes()
model.kinetic_ions.set_save_data(n_markers=1.0)

# propagator options
model.propagators.push_bxe.options = model.propagators.push_bxe.Options(tol=1e-5)
model.propagators.push_parallel.options = model.propagators.push_parallel.Options(tol=1e-5)

# initial conditions (background + perturbation)
perturbation = None
background = maxwellians.GyroMaxwellian2D(n=(1.0, perturbation), equil=equil)

model.kinetic_ions.var.add_background(background)

In [ ]:
# generate two axes
fig, axs = plt.subplots(2, 1, figsize=(8, 16))
ax = axs[0]
ax_top = axs[1]

# min/max of field strength
equil.domain = domain
Bmax = np.max(equil.absB0(*eta_topview_2, squeeze_out=True))
Bmin = np.min(equil.absB0(*eta_topview_1, squeeze_out=True))
levels = np.linspace(Bmin, Bmax, 51)

# absolute magnetic field at phi = 0
im = ax.contourf(x_pol, z_pol, equil.absB0(*eta_poloidal, squeeze_out=True), levels=levels)

# absolute magnetic field at Z = 0
im_top = ax_top.contourf(x_top1, y_top1, equil.absB0(*eta_topview_1, squeeze_out=True), levels=levels)
ax_top.contourf(x_top2, y_top2, equil.absB0(*eta_topview_2, squeeze_out=True), levels=levels)

# last closed flux surface, poloidal
ax.plot(x_pol[-1], z_pol[-1], color="k")

# last closed flux surface, toroidal
ax_top.plot(x_top1[-1], y_top1[-1], color="k")
ax_top.plot(x_top2[-1], y_top2[-1], color="k")

# limiter, poloidal
ax.plot(equil.limiter_pts_R, equil.limiter_pts_Z, "tab:orange")
ax.axis("equal")
ax.set_xlabel("R")
ax.set_ylabel("Z")
ax.set_title("abs(B) at $\phi=0$")
fig.colorbar(im)
# limiter, toroidal
limiter_Rmax = np.max(equil.limiter_pts_R)
limiter_Rmin = np.min(equil.limiter_pts_R)

thetas = 2 * np.pi * e2
limiter_x_max = limiter_Rmax * np.cos(thetas)
limiter_y_max = -limiter_Rmax * np.sin(thetas)
limiter_x_min = limiter_Rmin * np.cos(thetas)
limiter_y_min = -limiter_Rmin * np.sin(thetas)

ax_top.plot(limiter_x_max, limiter_y_max, "tab:orange")
ax_top.plot(limiter_x_min, limiter_y_min, "tab:orange")
ax_top.axis("equal")
ax_top.set_xlabel("x")
ax_top.set_ylabel("y")
ax_top.set_title("abs(B) at $Z=0$")
fig.colorbar(im_top);

In [ ]:
time_opts = Time(dt=0.1, Tend=100, split_algo="Strang")

verbose = False

main.run(
    model,
    params_path=None,
    env=env,
    base_units=base_units,
    time_opts=time_opts,
    domain=domain,
    equil=equil,
    grid=grid,
    derham_opts=derham_opts,
    verbose=verbose,
)

In [ ]:
import os

from struphy import main

path = os.path.join(os.getcwd(), "sim_1")
main.pproc(path)

simdata = main.load_data(path)

In [ ]:
orbits = simdata.orbits["kinetic_ions"]

Nt = simdata.Nt["kinetic_ions"]
Np = simdata.Np["kinetic_ions"]

In [ ]:
import math

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

dt = time_opts.dt
Tend = time_opts.Tend

for i in range(Np):
    r = np.sqrt(orbits[:, i, 0] ** 2 + orbits[:, i, 1] ** 2)
    # poloidal
    ax.scatter(r, orbits[:, i, 2], c=colors[i % 4], s=1)
    # top view
    ax_top.scatter(orbits[:, i, 0], orbits[:, i, 1], c=colors[i % 4], s=1)

ax.set_title(f"{math.ceil(Tend / dt)} time steps")
ax_top.set_title(f"{math.ceil(Tend / dt)} time steps")
fig